# Data Cleasing Task

Jerry Wang

## 1. Importing and Inspecting

In [256]:
# Import packages
import pandas as pd

In [257]:
# Read in raw data
data_raw = pd.read_excel('Data/Input/messy_phol.xlsx')

data_raw.head()

,region,date,name,comment
0,ACT,2014-01-01,New Years Day,None
1,ACT,2014-01-27,Australia Day Sub,None
2,ACT,2014-03-10,Canberra Day -RD,None
3,ACT,2014-04-18,Good Friday,None
4,ACT,2014-04-19,Easter Saturday,None


In [258]:
data_raw.dtypes

region             object
date       datetime64[ns]
name               object
comment            object
dtype: object

We see that `date` field is already in the required datetime type.

There is no data for QLD.

In [259]:
data_raw['region'].unique()

array(['ACT', 'NSW', 'NT', 'SA', 'TAS', nan, 'VIC', 'WA'], dtype=object)

Data at index 470 and 662 has missing values.

In [260]:
print(data_raw[data_raw.isnull().any(axis=1)])

    region       date                    name comment
470    NaN 2019-04-25               ANZAC Day    None
662     WA        NaT  Queens Birthday WA -RD    None


The names of public holidays are very messy.

In [261]:
print(data_raw['name'].value_counts())

Christmas Day                                     63
Easter Monday                                     62
Good Friday                                       58
ANZAC Day                                         50
New Years Day                                     47
                                                  ..
Boxing Day / Proclamation Day (additional day)     1
Australia Day 26th                                 1
good Friday - type c                               1
Australia Day SUB                                  1
Easter MON                                         1
Name: name, Length: 72, dtype: int64


Look for duplication in the data.

In [262]:
data_raw.loc[data_raw.duplicated(keep=False), :]

,region,date,name,comment
573,VIC,2019-12-25,Christmas Day,None
774,VIC,2019-12-25,Christmas Day,None


## 2. Data Cleansing

### 2.0 Duplicate Data

We simply remove the duplicated data row.

In [263]:
data_raw.drop_duplicates(inplace=True)

### 2.1 Missing Data

There is data missing at index 470 and 662, and the data is roughly ordered by state and date. Therefore:
1. We inspect the data points around index 470 and 662
2. We can also inspect data with similar dates / names
3. We then fill in based on the context

#### 2.1.1 Missing Region

Here we print out both the rows around the missing data, and other rows containing the same date.

In [264]:
print(data_raw.iloc[465:475, :])

    region       date                           name comment
465    TAS 2019-01-01                  New Years Day    None
466    TAS 2019-01-28              Australia Day Sub    None
467    TAS 2019-03-11       Labour Day VIC - TAS -RD    None
468    TAS 2019-04-19                    Good Friday    None
469    TAS 2019-04-22                  Easter Monday    None
470    NaN 2019-04-25                      ANZAC Day    None
471    TAS 2019-06-10     Queens Birthday exc WA -RD    None
472    TAS 2019-12-25                  Christmas Day    None
473    TAS 2019-12-26  Boxing Day - Proclamation Day    None
474    TAS 2020-01-01                  New Years Day    None


In [265]:
print(data_raw.loc[data_raw['date'] == '2019-04-25', :])

    region       date       name comment
74     ACT 2019-04-25  ANZAC Day    None
175    NSW 2019-04-25  ANZAC Day    None
273     NT 2019-04-25  ANZAC Day    None
382     SA 2019-04-25  ANZAC Day    None
470    NaN 2019-04-25  ANZAC Day    None
569    VIC 2019-04-25  ANZAC Day    None
649     WA 2019-04-25  ANZAC Day    None


We see that the missing data is most likely for TAS, so we replace it with 'TAS'.

In [266]:
data_raw.at[470, 'region'] = 'TAS'

Checking that the missing value has been replaced.

In [267]:
print(data_raw.loc[470])

region                     TAS
date       2019-04-25 00:00:00
name                 ANZAC Day
comment                   None
Name: 470, dtype: object


#### 2.1.2 Missing Date

Here we print out both the rows around the missing data, and other rows containing the same holiday.

In [268]:
print(data_raw.iloc[660:665, :])

print(data_raw[(data_raw['region']=='WA') & data_raw['name'].str.contains('Queens')])

    region       date                           name comment
660     WA 2020-04-27                  ANZAC Day Add    None
661     WA 2020-06-01          Western Australia Day    None
662     WA        NaT         Queens Birthday WA -RD    None
663     WA 2020-12-25                  Christmas Day    None
664     WA 2020-12-26  Boxing Day - Proclamation Day    None
    region       date                    name comment
597     WA 2014-09-29  Queens Birthday WA -RD    None
608     WA 2015-09-28  Queens Birthday WA -RD    None
619     WA 2016-09-26  Queens Birthday WA -RD    None
631     WA 2017-09-25  Queens Birthday WA -RD    None
641     WA 2018-09-24  Queens Birthday WA -RD    None
651     WA 2019-09-30  Queens Birthday WA -RD    None
662     WA        NaT  Queens Birthday WA -RD    None


We observe that the missing date is most likely 2020's Queen's Birthday for WA. A quick Google search shows the date as 28/09/2020.

In [269]:
data_raw.at[662, 'date'] = '2020-09-28'

Checking that the missing value has been replaced.

In [270]:
print(data_raw.loc[662])

region                         WA
date          2020-09-28 00:00:00
name       Queens Birthday WA -RD
comment                      None
Name: 662, dtype: object


### 2.2 Add Weekday and Week Number

We use the built in pandas functions

In [271]:
data_raw['week_day'] = data_raw['date'].dt.day_name()
data_raw['week_number'] = data_raw['date'].dt.isocalendar().week

### 2.3 Public Holiday Names Cleanup

Inspecting the full list of unique public holiday names, we can see that there is a lot of suffixes that needs to be removed.

In [272]:
print(sorted(data_raw['name'].unique()))

['AFL Grand Final - Friday -RD', 'ANZ Day - additional day declared', 'ANZAC Day', 'ANZAC Day Add', 'Adelaide Cup -RD', 'Adelaide Cup Day', 'Anzac Day', 'Anzac Day (additional day)', 'Australia Day', 'Australia Day 26th', 'Australia Day ADD SA', 'Australia Day SUB', 'Australia Day Sub', 'Bank Holiday', 'Boxing Day', 'Boxing Day (additional day)', 'Boxing Day - Add', 'Boxing Day - Proclamation Day', 'Boxing Day - Proclamation Day Sub', 'Boxing Day / Proclamation Day', 'Boxing Day / Proclamation Day (additional day)', 'Boxing Day Add', 'Boxing Day Sub', 'Canberra Day', 'Canberra Day -RD', 'Christmas (additional day)', 'Christmas Day', 'Christmas Day Add', 'Christmas Eve', 'Christmas Eve -RD', 'Christmas Eve 7pm - midnight', 'Easter MON', 'Easter Monday', 'Easter Saturday', 'Easter Saturday - the Saturday following Good Friday', 'Easter Sunday', 'Easter Tuesday', 'Eight Hours Day', 'Family and Community Day', 'Friday before AFL Grand Final', 'Good Friday', 'Good Friday - type a', 'Good Fr

We attempt to clean up public holiday names by:
1. Removing all trailing '-RD'
2. Replace all 'Add' with '(additional day)'
3. Remove other text after the word 'day' if there's no '(additional day)'
4. Replace 'Anzac' with 'ANZAC'
5. Remove apostrophy

Function for step 3

In [273]:
def suffix_strip(phol_string):
    """Removes any string after the word 'day' unless it's part of 'additional day'

    Args:
    phol_string: (string) public holiday name 
    """
    if 'add' in phol_string.lower():
        return phol_string
    elif 'day' in phol_string.lower():
        return phol_string[:phol_string.lower().index('day') + 3]
    else:
        return phol_string

Function for other 4 steps

In [274]:
def substring_replace(phol_string, replace_dict):
    """Removes substring according to dictionary

    Args:
    phol_string: (string) public holiday name
    replace_dict: (dictionary) values to replace
    """
    for key, value in replace_dict.items():
        phol_string = phol_string.replace(key, value)
    return phol_string

In [275]:
phol_cleanup = {' -RD': '',
                'Add': '(additional day)',
                'Anzac': 'ANZAC',
                "'": ''
                }

data_raw['name'] = data_raw['name'].apply(lambda row: substring_replace(row, phol_cleanup))

data_raw['name'] = data_raw['name'].apply(lambda row: suffix_strip(row))

print(sorted(data_raw['name'].unique()))

['AFL Grand Final - Friday', 'ANZ Day - additional day declared', 'ANZAC Day', 'ANZAC Day (additional day)', 'Adelaide Cup', 'Adelaide Cup Day', 'Australia Day', 'Australia Day ADD SA', 'Bank Holiday', 'Boxing Day', 'Boxing Day (additional day)', 'Boxing Day - (additional day)', 'Boxing Day / Proclamation Day (additional day)', 'Canberra Day', 'Christmas (additional day)', 'Christmas Day', 'Christmas Day (additional day)', 'Christmas Eve', 'Christmas Eve 7pm - midnight', 'Easter MON', 'Easter Monday', 'Easter Saturday', 'Easter Sunday', 'Easter Tuesday', 'Eight Hours Day', 'Family and Community Day', 'Friday', 'Good Friday', 'Labour Day', 'May Day', 'Melbourne Cup', 'Melbourne Cup Day', 'NT Picnic Day', 'New Years Day', 'New Years Day (additional day)', 'New Years Day Sub (additional day)', 'New Years Eve', 'New Years Eve 7pm - midnight', 'Picnic Day', 'Queens Birthday', 'Reconciliation Day', 'Saturday', 'Western Australia Day', 'good Friday']


There is still multiple messy or inconsistent holiday names, so we pick them out to investigate separately.

In [276]:
investigate_list = ['ANZ Day - additional day declared',
                    'Adelaide Cup',
                    'Australia Day ADD SA',
                    'Boxing Day - (additional day)',
                    'Boxing Day / Proclamation Day (additional day)',
                    'Christmas (additional day)',
                    'Christmas Eve 7pm - midnight',
                    'Easter MON',
                    'Easter Tuesday',
                    'Eight Hours Day',
                    'Family and Community Day',
                    'Friday',
                    'Labour Day',
                    'May Day',
                    'Melbourne Cup',
                    'NT Picnic Day',
                    "New Years Eve 7pm - midnight",
                    'Saturday',
                    'good Friday']

for phol in investigate_list:
    print(phol, '-'*50)
    print(data_raw[data_raw['name'].str.startswith(phol)], '\n')


ANZ Day - additional day declared --------------------------------------------------
   region       date                               name comment week_day  \
88    ACT 2020-04-27  ANZ Day - additional day declared    None   Monday   

    week_number  
88           18   

Adelaide Cup --------------------------------------------------
    region       date              name comment week_day  week_number
311     SA 2014-03-10      Adelaide Cup    None   Monday           11
324     SA 2015-03-09      Adelaide Cup    None   Monday           11
337     SA 2016-03-14      Adelaide Cup    None   Monday           11
352     SA 2017-03-13      Adelaide Cup    None   Monday           11
365     SA 2018-03-12      Adelaide Cup    None   Monday           11
378     SA 2019-03-11      Adelaide Cup    None   Monday           11
392     SA 2020-03-09      Adelaide Cup    None   Monday           11
405     SA 2021-03-08  Adelaide Cup Day    None   Monday           10
724     SA 2022-03-14  Adelaid

We then rename these individual cases to be in line with the general case.

In [277]:
phol_replacement = {'ANZ Day - additional day declared': 'ANZAC Day (additional day)',
                    'Adelaide Cup': 'Adelaide Cup Day',
                    'Australia Day ADD SA': 'Australia Day (additional day)',
                    'Boxing Day - (additional day)': 'Boxing Day (additional day)',
                    'Boxing Day / Proclamation Day (additional day)': 'Boxing Day (additional day)',
                    'Christmas (additional day)': 'Christmas Day (additional day)',
                    'Christmas Eve 7pm - midnight': 'Christmas Eve',
                    'Easter MON': 'Easter Monday',
                    'Easter Tuesday': 'Easter Tuesday',
                    'Eight Hours Day': 'Eight Hours Day',
                    'Family and Community Day': 'Family and Community Day',
                    'Friday': 'AFL Grand Final - Friday',
                    'Labour Day': 'Labour Day',
                    'May Day': 'May Day',
                    'Melbourne Cup': 'Melbourne Cup Day',
                    'NT Picnic Day': 'Picnic Day',
                    "New Years Eve 7pm - midnight": 'New Years Eve',
                    'Saturday': 'Easter Saturday',
                    'good Friday': 'Good Friday'
                    }

Here we replace based on the dictionary above

In [278]:
data_raw['name'].replace(to_replace=phol_replacement, inplace=True)

print(sorted(data_raw['name'].unique()))

['AFL Grand Final - Friday', 'ANZAC Day', 'ANZAC Day (additional day)', 'Adelaide Cup Day', 'Australia Day', 'Australia Day (additional day)', 'Bank Holiday', 'Boxing Day', 'Boxing Day (additional day)', 'Canberra Day', 'Christmas Day', 'Christmas Day (additional day)', 'Christmas Eve', 'Easter Monday', 'Easter Saturday', 'Easter Sunday', 'Easter Tuesday', 'Eight Hours Day', 'Family and Community Day', 'Good Friday', 'Labour Day', 'May Day', 'Melbourne Cup Day', 'New Years Day', 'New Years Day (additional day)', 'New Years Day Sub (additional day)', 'New Years Eve', 'Picnic Day', 'Queens Birthday', 'Reconciliation Day', 'Western Australia Day']


In [279]:
data_raw

,region,date,name,comment,week_day,week_number
0,ACT,2014-01-01,New Years Day,None,Wednesday,1
1,ACT,2014-01-27,Australia Day,None,Monday,5
2,ACT,2014-03-10,Canberra Day,None,Monday,11
3,ACT,2014-04-18,Good Friday,None,Friday,16
4,ACT,2014-04-19,Easter Saturday,None,Saturday,16
...,...,...,...,...,...,...
769,WA,2022-09-26,Queens Birthday,None,Monday,39
770,WA,2022-12-25,Christmas Day,None,Sunday,51
771,WA,2022-12-26,Boxing Day,None,Monday,52
772,WA,2022-12-27,Christmas Day (additional day),None,Tuesday,52


Rename the columns and save to a copy 

In [280]:
data_cleansed = data_raw.copy(deep=True)
data_cleansed.drop(columns='comment', inplace=True)

data_cleansed.columns = ['phol_location',
                         'public_holiday_date',
                         'public_holiday',
                         'week_day',
                         'week_number']

data_cleansed.head()

,phol_location,public_holiday_date,public_holiday,week_day,week_number
0,ACT,2014-01-01,New Years Day,Wednesday,1
1,ACT,2014-01-27,Australia Day,Monday,5
2,ACT,2014-03-10,Canberra Day,Monday,11
3,ACT,2014-04-18,Good Friday,Friday,16
4,ACT,2014-04-19,Easter Saturday,Saturday,16


Here we take a subset of the data for the in-scope period.  

In [281]:
data_inscope = data_cleansed.loc[(data_cleansed['public_holiday_date'] >= '2019-01-01') & 
                                 (data_cleansed['public_holiday_date'] <= '2021-12-31'),
                                 :
                                 ].copy(deep=True)

data_inscope.head()

,phol_location,public_holiday_date,public_holiday,week_day,week_number
67,ACT,2019-01-01,New Years Day,Tuesday,1
68,ACT,2019-01-28,Australia Day,Monday,5
69,ACT,2019-03-11,Canberra Day,Monday,11
70,ACT,2019-04-19,Good Friday,Friday,16
71,ACT,2019-04-20,Easter Saturday,Saturday,16


### 2.4 Public Holiday Start Time

We create the start and end time columns by copying over the `public_holiday_date` column first.

We then create a copy of the smaller subset of the in-scope data, for those holidays that have a 7PM start time instead of 12AM start time.

In [282]:
data_inscope['public_holiday_start'] = data_inscope['public_holiday_date']
data_inscope['public_holiday_end'] = data_inscope['public_holiday_date']

# Removing the time components so the format when output to CSV is correct
data_inscope['public_holiday_date'] = data_inscope['public_holiday_date'].dt.date

In [283]:
partial_holidays = data_inscope.loc[(data_inscope['phol_location'].isin(['NT', 'SA'])) & 
                                    (data_inscope['public_holiday'].isin(['Christmas Eve', 'New Years Eve'])),
                                    :
                                    ].copy(deep=True)

We replace the time with 19:00 or 7PM.

In [284]:
partial_holidays.loc[:, 'public_holiday_start'] = partial_holidays.loc[:, 'public_holiday_start'].apply(
                                                      lambda x: x.replace(hour=19, minute=0)
                                                      )

print(partial_holidays)

    phol_location public_holiday_date public_holiday  week_day  week_number  \
277            NT          2019-12-24  Christmas Eve   Tuesday           52   
280            NT          2019-12-31  New Years Eve   Tuesday            1   
290            NT          2020-12-24  Christmas Eve  Thursday           52   
293            NT          2020-12-31  New Years Eve  Thursday           53   
303            NT          2021-12-24  Christmas Eve    Friday           51   
308            NT          2021-12-31  New Years Eve    Friday           52   
385            SA          2019-12-24  Christmas Eve   Tuesday           52   
388            SA          2019-12-31  New Years Eve   Tuesday            1   
399            SA          2020-12-24  Christmas Eve  Thursday           52   
402            SA          2020-12-31  New Years Eve  Thursday           53   
413            SA          2021-12-24  Christmas Eve    Friday           51   
418            SA          2021-12-31  New Years Eve

We update the original `data_inscope` table, and check that the values have been successfully updated

In [285]:
data_inscope.update(partial_holidays)

data_inscope.loc[(data_inscope['phol_location'].isin(['NT', 'SA'])) & 
                 (data_inscope['public_holiday'].isin(['Christmas Eve', 'New Years Eve'])),
                 :
                 ]

,phol_location,public_holiday_date,public_holiday,week_day,week_number,public_holiday_start,public_holiday_end
277,NT,2019-12-24,Christmas Eve,Tuesday,52,2019-12-24 19:00:00,2019-12-24
280,NT,2019-12-31,New Years Eve,Tuesday,1,2019-12-31 19:00:00,2019-12-31
290,NT,2020-12-24,Christmas Eve,Thursday,52,2020-12-24 19:00:00,2020-12-24
293,NT,2020-12-31,New Years Eve,Thursday,53,2020-12-31 19:00:00,2020-12-31
303,NT,2021-12-24,Christmas Eve,Friday,51,2021-12-24 19:00:00,2021-12-24
308,NT,2021-12-31,New Years Eve,Friday,52,2021-12-31 19:00:00,2021-12-31
385,SA,2019-12-24,Christmas Eve,Tuesday,52,2019-12-24 19:00:00,2019-12-24
388,SA,2019-12-31,New Years Eve,Tuesday,1,2019-12-31 19:00:00,2019-12-31
399,SA,2020-12-24,Christmas Eve,Thursday,52,2020-12-24 19:00:00,2020-12-24
402,SA,2020-12-31,New Years Eve,Thursday,53,2020-12-31 19:00:00,2020-12-31


### 2.5 Public Holiday End Time

Here we simply change all public holiday end time to 23:59:59.

In [286]:
data_inscope.loc[:, 'public_holiday_end'] = data_inscope.loc[:, 'public_holiday_end'].apply(
                                                lambda x: x.replace(hour=23, minute=59, second=59)
                                                )

In [287]:
print(data_inscope.head())

   phol_location public_holiday_date   public_holiday  week_day week_number  \
67           ACT          2019-01-01    New Years Day   Tuesday           1   
68           ACT          2019-01-28    Australia Day    Monday           5   
69           ACT          2019-03-11     Canberra Day    Monday          11   
70           ACT          2019-04-19      Good Friday    Friday          16   
71           ACT          2019-04-20  Easter Saturday  Saturday          16   

   public_holiday_start  public_holiday_end  
67           2019-01-01 2019-01-01 23:59:59  
68           2019-01-28 2019-01-28 23:59:59  
69           2019-03-11 2019-03-11 23:59:59  
70           2019-04-19 2019-04-19 23:59:59  
71           2019-04-20 2019-04-20 23:59:59  


### 2.6 Final Check

Final check for missing data

In [288]:
data_inscope[data_inscope.isnull().any(axis=1)]

,phol_location,public_holiday_date,public_holiday,week_day,week_number,public_holiday_start,public_holiday_end


Final check for duplicates

In [289]:
data_inscope.loc[data_inscope.duplicated(keep=False), :]

,phol_location,public_holiday_date,public_holiday,week_day,week_number,public_holiday_start,public_holiday_end
185,NSW,2020-04-13,Easter Monday,Monday,16,2020-04-13,2020-04-13 23:59:59
773,NSW,2020-04-13,Easter Monday,Monday,16,2020-04-13,2020-04-13 23:59:59


The duplication was from 'Easter MON' and 'Easter Monday' name changes, we therefore remove them.

In [290]:
data_inscope.drop_duplicates(inplace=True)

### 2.7 Saving to CSV

In [291]:
data_inscope.to_csv('Data/Output/cleaned_phol.csv', index=False)